In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from matplotlib.widgets import Slider, Button
import astropy.units as u
import astropy.constants as uc

from ipywidgets import interact
import ipywidgets as widgets

plt.style.use("nonos.default")

from kintomo.api import Sculpture, Shape, Velocity, Grid, Geometry, projection_deposition_visible

In [ ]:
num_points = int(5e5)
x0, y0, z0 = (2 * np.random.rand(num_points) - 1 for _ in range(3))
unit_l = 100.0*u.au
unit_m = 1.0*u.M_sun
unit_v = np.sqrt(uc.G*unit_m/unit_l).to(u.km/u.s)

cylinder = (x0**2 + y0**2 < 1.0**2) & (abs(z0) < 0.02)
sculpture = Sculpture(x=x0, y=y0, z=z0).carve(
    shape=Shape(cylinder)
).to(unit=unit_l)
r, phi, z = sculpture.cylindrical_coordinates()
velocity = Velocity.keplerian(r=r/unit_l).to_cartesian(phi=phi).to(unit=unit_v)

nx, ny, nz = (65, 65, 33)
grid = Grid.encompass(
    sculpture=sculpture, 
    dimension=(nx, ny, nz),
)

In [22]:
vshift_extent = velocity.shift(get="mean")
dv = 0.02*vshift_extent

def plot_vmap(inclination, vshift):
    v_visible = projection_deposition_visible(
        grid=grid,
        sculpture=sculpture,
        velocity=velocity,
        angle=inclination,
        method="ngp",
    )

    fig, ax = plt.subplots(figsize=(10,10))
    im = ax.pcolormesh(
        "x",
        "y",
        v_visible.T,
        data=grid.cell_edges,
        vmin=-vshift_extent, 
        vmax=vshift_extent,
        cmap="RdBu_r",
    )
    contours = ax.contour(
        "x", 
        "y", 
        v_visible.T, 
        data=grid.cell_centers, 
        levels=np.linspace(vshift-dv,vshift+dv+0.003,10), 
        colors="k", 
        linestyles="solid"
    )
    ax.set(
        aspect="equal", 
        ylim=((3/5)*grid.yedge.min(), (3/5)*grid.yedge.max()), 
        xlabel=f"x [{sculpture.get_unit}]", 
        ylabel=f"y [{sculpture.get_unit}]"
    )

    from mpl_toolkits.axes_grid1 import make_axes_locatable

    divider = make_axes_locatable(ax)
    cax = divider.append_axes("right", size="5%", pad=0.05)
    cbar = fig.colorbar(
        im, cax=cax, orientation="vertical"
    )
    cbar.set_label(rf"$v_{{\rm proj}}$ [{velocity.get_unit}]")
    plt.show()

inclination_widget = widgets.FloatSlider(value = 40,
                          min = -90, 
                          max = 90,
                          step = 1.0,
                          description = "inclination",
                          continuous_update = False)

vshift_widget = widgets.FloatSlider(value = 0,
                          min = -vshift_extent, 
                          max = vshift_extent,
                          step = 0.1,
                          description = "vshift",
                          continuous_update = False)

interact(plot_vmap, inclination=inclination_widget, vshift=vshift_widget)

interactive(children=(FloatSlider(value=40.0, continuous_update=False, description='inclination', max=90.0, mi…

<function __main__.plot_vmap(inclination, vshift)>

In [ ]:
vshift_extent = velocity.shift(get="mean")

def get_projected_vz(*, velocity:Velocity, angle:float):
    projected_velocity = velocity.projected_on_sky(angle=angle)
    v3 = projected_velocity.v3
    if (u.get_physical_type(projected_velocity.v3)=="velocity"):
        v3 = projected_velocity.v3.value
    return v3

def plot_spectrum(inclination):
    projected_vz = get_projected_vz(velocity=velocity, angle=inclination)
    fig, ax = plt.subplots(figsize=(12,7))
    style = {"facecolor": "none", "edgecolor": "black", "linewidth": 1}
    ax.hist(
        projected_vz,
        bins=np.linspace(-vshift_extent,vshift_extent,200), 
        density=True, 
        **style
    )
    ax.set(
        xlabel=f"v [{velocity.get_unit}]",
        ylabel="Probability density",
        xlim=(-vshift_extent,vshift_extent),
    )

inclination_widget = widgets.FloatSlider(value = 33,
                          min = -90, 
                          max = 90,
                          step = 1.0,
                          description = "inclination",
                          continuous_update = False)

interact(plot_spectrum, inclination=inclination_widget)

interactive(children=(FloatSlider(value=33.0, continuous_update=False, description='inclination', max=90.0, mi…

<function __main__.plot_spectrum(inclination)>

In [ ]:
from skimage import measure
vshift_extent = velocity.shift(get="mean")
dv = 0.02*vshift_extent

def get_indices_of_sign_change(lst):
    arr = np.array(lst)
    sign_changes = np.where(np.diff(np.sign(arr)) != 0)[0]
    return sign_changes.tolist()

def plot_vmap_pv(inclination, vshift, altitude):
    v_visible = projection_deposition_visible(
        grid=grid,
        sculpture=sculpture,
        velocity=velocity,
        angle=inclination,
        method="ngp",
    )

    fig, ax = plt.subplots(nrows=2, figsize=(10,10))
    im = ax[0].pcolormesh(
        "x",
        "y",
        v_visible.T,
        data=grid.cell_edges,
        vmin=-vshift_extent, 
        vmax=vshift_extent,
        cmap="RdBu_r",
    )
    contours = ax[0].contour(
        "x", 
        "y", 
        v_visible.T, 
        data=grid.cell_centers, 
        levels=np.linspace(vshift-dv,vshift+dv+0.003,10), 
        colors="k", 
        linestyles="solid"
    )
    ax[0].axhline(y=altitude, c="green")

    ax[1].set(
        xlabel=f"x [{sculpture.get_unit}]", 
        ylabel=rf"$v_{{\rm proj}}$ [{velocity.get_unit}]",
        xlim=((3/5)*grid.xedge.min(), (3/5)*grid.xedge.max()), 
        ylim=(-vshift_extent, vshift_extent)
    )

    for vi in np.linspace(-vshift_extent,vshift_extent,100):
        contours_sk0 = measure.find_contours(v_visible, level=vi)
        if len(contours_sk0)!=0:
            for icont in contours_sk0:
                ixseg, iyseg = np.rint(icont.T).astype(int)
                xseg, yseg = (grid.cell_centers[k][ik] for k, ik in zip(["x","y"],[ixseg,iyseg]))
                isect = get_indices_of_sign_change(np.array(yseg)-altitude)
                xisect = xseg[isect]
                visect = [vi]*len(xseg[isect])
                ax[1].scatter(
                    xisect,
                    visect,
                    c="k"
                )

    contours_sk = measure.find_contours(v_visible, level=vshift)
    if len(contours_sk)!=0:
        for icont in contours_sk:
            ixseg, iyseg = np.rint(icont.T).astype(int)
            xseg, yseg = (grid.cell_centers[k][ik] for k, ik in zip(["x","y"],[ixseg,iyseg]))

            ax[0].scatter(xseg, yseg, c="green", s=5, zorder=3)

            isect = get_indices_of_sign_change(np.array(yseg)-altitude)
            xisect = xseg[isect]
            yisect = yseg[isect]
            visect = [vshift]*len(xseg[isect])

            ax[0].scatter(
                xisect, 
                yisect, 
                s=50, 
                c="yellow", 
                zorder=4
            )
            ax[1].scatter(
                xisect,
                visect,
                c="yellow"
            )

    ax[0].set(
        aspect="equal", 
        ylim=((3/5)*grid.yedge.min(), (3/5)*grid.yedge.max()), 
        xlabel=f"x [{sculpture.get_unit}]", 
        ylabel=f"y [{sculpture.get_unit}]"
    )

    from mpl_toolkits.axes_grid1 import make_axes_locatable

    divider = make_axes_locatable(ax[0])
    cax = divider.append_axes("right", size="5%", pad=0.05)
    cbar = fig.colorbar(
        im, cax=cax, orientation="vertical"
    )
    cbar.set_label(rf"$v_{{\rm proj}}$ [{velocity.get_unit}]")
    plt.show()

inclination_widget = widgets.FloatSlider(value = 0,
                          min = -90, 
                          max = 90,
                          step = 1.0,
                          description = "inclination",
                          continuous_update = False)

vshift_widget = widgets.FloatSlider(value = 0,
                          min = -vshift_extent, 
                          max = vshift_extent,
                          step = 0.1,
                          description = "vshift",
                          continuous_update = False)

altitude_widget = widgets.FloatSlider(value = 0,
                          min = -grid.yedge.max()/2, 
                          max = grid.yedge.max()/2,
                          step = 0.1,
                          description = "altitude",
                          continuous_update = False)

interact(plot_vmap_pv, inclination=inclination_widget, vshift=vshift_widget, altitude=altitude_widget)


interactive(children=(FloatSlider(value=0.0, continuous_update=False, description='inclination', max=90.0, min…

<function __main__.plot_vmap_pv(inclination, vshift, altitude)>